In [1]:
import trading as trd

Using Theano backend.


In [2]:
# relaod module after modifying it
import importlib
importlib.reload(trd.stock_history)
importlib.reload(trd.nn)
importlib.reload(trd.testing)
importlib.reload(trd)

<module 'trading' from 'C:\\Users\\hamza\\Documents\\code\\python\\cs229\\src\\trading\\__init__.py'>

## Portfolio Setup

In [4]:
model_dirs = ['30days_pen_return_l=0',
             '30days_pen_return_l=0_5',
             '30days_sharpe_reward']

In [5]:
trd.test_models(model_dirs)

running model 30days_pen_return_l=0 (1 of 3)
running model 30days_pen_return_l=0_5 (2 of 3)
running model 30days_sharpe_reward (3 of 3)
done.
